In [5]:
import requests
from datetime import date
from calendar import monthrange

In [42]:
# https://files.airnowtech.org/airnow/2016/20161101/peak_aqi_usa.jpg
image_url_base = "https://files.airnowtech.org/airnow/{}/{}/peak_aqi_usa.jpg"

In [ ]:
s = requests.session()
numdays = 365
base = datetime.datetime.today()
date_list = [base - datetime.timedelta(days=x) for x in range(0, numdays)]
for date in date_list:
    ymd = "{}{:02}{:02}".format(date.year, date.month, date.day)
    url = image_url_base.format(date.year, ymd)
    download_file = "pm25_images/{}.jpg".format(ymd)
    import os.path
    if os.path.isfile(download_file):
        continue
    r = s.get(url, stream=True)
    if r.status_code == 200:
        with open(download_file, 'wb') as f:
            for chunk in r:
                f.write(chunk)